In [215]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [216]:
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go


In [217]:
df=pd.read_csv('../input/devfest21-competition/Train (1).csv')
df.head()

In [218]:
df.info()

In [219]:
df.shape

In [220]:
df.describe()

In [221]:
df.corr()

In [222]:
A=[]
B=[]
A1=[]
A2=[]
for i in range (28):
    x=df.loc[df['Administrative']==i].id.count()
    y=df.loc[df['Administrative']==i].loc[df['Revenue']==True].id.count()
    A+=[i]
    B+=[y/x]
    A1+=[x]
    A2+=[y]
    
    
fig = go.Figure(data=[go.Scatter(
    x=A,
    y=B,
    mode='markers',
    marker=dict(
        color=A1,
        size=A2,
        showscale=True
        )
)])
fig.show()

In [223]:
E=[]
F=[]
E1=[]
E2=[]
for i in range (0,710,10):
    E+=[i]
    x=0
    y=0
    for j in range (8731):
        
        if ((df['ProductRelated'][j]>=i) and (df['ProductRelated'][j]<i+20)):
            x+=1
            if (df['Revenue'][j]==True):
                y+=1
    if (x!=0):
        F+=[y/x]
        E1+=[x]
        E2+=[y]
    else :   
        E.remove(i)
        
fig = go.Figure(data=[go.Scatter(
    x=E,
    y=F,
    mode='markers',
    marker=dict(
        color=E1,
        size=(np.array(F)*100),
        showscale=True
        )
)])

fig.show()

In [224]:
G=[]
H=[]
H1=[]
H2=[]
for i in range (0,370,20):
    x=0
    y=0
    G+=[i]
    for j in range (8731):
        if (df['PageValues'][j]>i) and (df['PageValues'][j]<i+10):
            x=x+1
            if (df['Revenue'][j]==True):
                y+=1
    
    
    if (x!=0):
        H+=[y/x]
        H1+=[x] 
        H2+=[y]
    else :
        G.remove(i)
fig = go.Figure(data=[go.Scatter(
    x=G,
    y=H,
    mode='markers',
    marker=dict(  
        color=H1,
        size=(np.array(H)*100),
        showscale=True
        )
)])

fig.show()

In [225]:
I=[]
J=[]
I1=[]
I2=[]
for i in range (20):
    x=0
    y=0
    I+=[(i/100)]
    for j in range (8731):
        if ((df['BounceRates'][j]>(i/100)) and (df['BounceRates'][j]<((i/100)+0.01))):
            x=x+1
            if (df['Revenue'][j]==True):
                y+=1
    if (x!=0):
        J+=[y/x]
        I1+=[x] 
        I2+=[y]
    else :
        I.remove((i/100))
fig = go.Figure(data=[go.Scatter(
    x=I,
    y=J,
    mode='markers',
    marker=dict(
        color=I1,
        size=(np.array(J)*1000),
        showscale=True
        )
)])

fig.show()

In [226]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

In [227]:
dt=pd.read_csv('../input/devfest21-competition/Test (1).csv')
ds=pd.read_csv('../input/devfest21-competition/sample.csv')

In [228]:
df=df.dropna()
dt=dt.dropna()

In [229]:
dt.head()

In [230]:
del df['VisitorType']
del df['Month']
del dt['VisitorType']
del dt['Month']


In [231]:
dt.head()

In [232]:
print(X_test.shape)

In [233]:
y_test=ds['Revenue']
y_train=df['Revenue']
X_test=dt
X_train=df.iloc[:,:-1]  

In [234]:
# Define parameters: these will need to be tuned to prevent overfitting and underfitting
params = {
    "n_estimators": 100,  # Number of trees in the forest
    "max_depth": 10,  # Max depth of the tree
    "min_samples_split": 4,  # Min number of samples required to split a node
    "min_samples_leaf": 2,  # Min number of samples required at a leaf node
    "ccp_alpha": 0,  # Cost complexity parameter for pruning
    "random_state": 123,
}

# Create a RandomForestRegressor object with the parameters above
rf = RandomForestRegressor(**params)

# Train the random forest on the train set
rf = rf.fit(X_train, y_train)
# Predict the outcomes on the test set
y_pred = rf.predict(X_test)
y_pred=np.round(np.array(y_pred))
print(y_pred)
print(y_test)

In [235]:
y_test1=y_test.iloc[0:5]
y_pred1=y_pred[0:5]
print(y_pred1)

In [236]:

print("Mean Absolute Error:", metrics.mean_absolute_error(y_test1, y_pred1))
print("Mean Squared Error:", metrics.mean_squared_error(y_test1, y_pred1))
print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test1, y_pred1)))

In [237]:
# Create a sorted Series of features importances
importances_sorted = pd.Series(
    data=rf.feature_importances_, index=X_train.columns
).sort_values()

# Plot a horizontal barplot of importances_sorted
importances_sorted.plot(kind="barh")
plt.title("Features Importances")
plt.show()

In [238]:
import sklearn
sklearn.metrics.f1_score(y_test1,y_pred1)

In [239]:
X_test.reset_index(inplace=True)
X_test['Revenue'] = y_pred
submission=X_test[['id','Revenue']]
submission.set_index('id',inplace=True)
submission.to_csv('submission.csv')

In [240]:
submission.head()